In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert pixel values to floats in the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Subtract the mean pixel value from each image
mean_pixel = x_train.mean(axis=(0,1,2), keepdims=True)
x_train -= mean_pixel
x_test -= mean_pixel

# Convert labels to one-hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define the ResNet block
def resnet_block(inputs, filters, stride, reg_factor):
    x = Conv2D(filters, (3, 3), strides=stride, padding='same', kernel_regularizer=l2(reg_factor))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), strides=1, padding='same', kernel_regularizer=l2(reg_factor))(x)
    x = BatchNormalization()(x)
    shortcut = inputs
    if stride != 1 or inputs.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='valid', kernel_regularizer=l2(reg_factor))(inputs)
        shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Define the ResNet model
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), strides=1, padding='same', kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = resnet_block(x, 64, 1, 1e-4)
x = resnet_block(x, 64, 1, 1e-4)
x = resnet_block(x, 64, 1, 1e-4)
x = resnet_block(x, 128, 2, 1e-4)
x = resnet_block(x, 128, 1, 1e-4)
x = resnet_block(x, 128, 1, 1e-4)
x = resnet_block(x, 256, 2, 1e-4)
x = resnet_block(x, 256, 1, 1e-4)
x = resnet_block(x, 256, 1, 1e-4)
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Flatten()(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))


In [ ]:
plt.plot(resnet.history['accuracy'], label='accuracy')
plt.plot(resnet.history['val_accuracy'], label = 'val_accuracy')
plt.title('ResNet')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)